In [1]:
!pip install -q pyomo

In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
chmod u+rx ipopt

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


#Ans 1 :

$\text{min} (x_l-x_f)^2 + (y_l-y_f)^2$

Since we just want to find the points, we sqaure the distance in the objective and minimize it. 

subject to :

$(x_l+200)^2+(y_l-200)^2 \leq50^2\\
x_f+y_f\leq0\\
x_f-y_f\leq200\\
x_f+y_f\geq-200\\
x_f-3y_f\geq0$

In [5]:
model = ConcreteModel()

In [6]:
model.xf = Var()
model.yf = Var()
model.xl = Var()
model.yl = Var()

In [7]:
model.objective = Objective(expr = (model.xl-model.xf)**2+(model.yl-model.yf)**2, sense = minimize)

In [8]:
model.constraints = ConstraintList()

In [9]:
model.constraints.add((model.xl+200)**2+(model.yl-200)**2 <= 2500)
model.constraints.add(model.xf+model.yf <= 0)
model.constraints.add(model.xf-model.yf <= 200)
model.constraints.add(model.xf+model.yf >= -200)
model.constraints.add(model.xf-3*model.yf >= 0)

In [10]:
model.pprint()

1 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

4 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    yf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    yl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (xl - xf)**2 + (yl - yf)**2

1 Con

In [11]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.objective())

print('\nDecision Variables')
print('xl = ', model.xl.value)
print('yl = ', model.yl.value)  
print('xf = ', model.xf.value)
print('yf = ', model.yf.value)  


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.026293039321899414
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  41201.7786158831

Decision Variables
xl =  -184.18861162009682
yl =  152.56583486031144
xf =  -120.00000000096625
yf =  -39.99999999699526


#Ans 3 :

The locations of the two ends will be :

$(x_l,y_l)=(-184.19,152.57)\\
(x_f,y_f)=(-120.00,-39.99)$

#Ans 4 :

We introduce two new variables $x_j$ and $y_j$ which denote the coordinates of the junction. We then change the objective function accordingly:

$\text{min} \sqrt{(x_l-x_j)^2 + (y_l-y_j)^2}+\sqrt{(x_j-x_f)^2 + (y_j-y_f)^2}+0.2\sqrt{(x_j+15)^2 + (y_j-20)^2}$

All the constraints remain the same.


In [12]:
model.xj = Var()
model.yj = Var()

In [13]:
model.objective.deactivate()

In [14]:
model.objective_1 = Objective(expr = sqrt((model.xl-model.xj)**2+(model.yl-model.yj)**2)+sqrt((model.xj-model.xf)**2+(model.yj-model.yf)**2)+0.2*sqrt((model.xj+15)**2+(model.yj-20)**2), sense = minimize)

In [15]:
model.pprint()

1 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

6 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value               : Upper : Fixed : Stale : Domain
        None :  None : -120.00000000096625 :  None : False : False :  Reals
    xj : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xl : Size=1, Index=None
        Key  : Lower : Value               : Upper : Fixed : Stale : Domain
        None :  None : -184.18861162009682 :  None : False : False :  Reals
    yf : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :  None : -39.99999999699526 :  None : False : False :  Reals
    yj : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  N

In [16]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.objective())

print('\nDecision Variables')
print('xl = ', model.xl.value)
print('yl = ', model.yl.value) 
print('xf = ', model.xf.value)
print('yf = ', model.yf.value)
print('xj = ', model.xj.value)
print('yj = ', model.yj.value)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.03089118003845215
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  43056.16649434129

Decision Variables
xl =  -175.06555970851505
yl =  156.66094472534294
xf =  -74.46804648782
yf =  -24.822682161914805
xj =  -79.72453251652577
yj =  -9.05322407544095


#Ans 6 :

The locations of the two ends will be :

$(x_l,y_l)=(-175.07,156.66)\\
(x_f,y_f)=(-74.47,-24.82)$

